# Modulation Classification Training (Fixed-point)

Welcome to the training notebook for the modulation classification model using Quantised-Aware Training (QAT).

The goal of this notebook is to train a Convolutional Neutral Network (CNN) model to learn how to classify modulation schemes using the DeepRFSoC dataset recorded with an AMD RFSoC device.

## Model Dimensions
The CNN model is a small, 4-layer network with two convolutional layers and two fully-connected layers.

<img src="./assets/networktopology.png" width="800" alt="Model Topology">

## Quantisation Parameters
Since we are aiming to implement our trained model onto the FPGA while quantising our weights and activations. We can help the model learn to adjust for the loss in precision due to the quantisation process.

Four models are trained and compared against eachother configured with varying weights quantisations.

| Model  | Weight Precision   | Activation Precision |
|--------|--------------------|----------------------|
| 16w16a | 16-bit Fixed-point | 16-bit Fixed-point   |
| 8w16a  | 8-bit Fixed-point  | 16-bit Fixed-point   |
| 4w16a  | 4-bit Fixed-point  | 16-bit Fixed-point   |
| 2w16a  | 2-bit Fixed-point  | 16-bit Fixed-point   |

We are varying the precision of our weights to analyse how low the precision can be pushed. The activation precisions are maintained at 16-bits as this is where our signal information is held. The AMD RFSoC receiver produces I/Q samples at 16-bit, so we have decided to maintain the precision of the information signal.

---

## Import Packages and Confirm GPU are available
Whether you are using CUDA or ROCm, confirm you can accelerate the training loops with a GPU. Otherwise, this can run on the CPU (slow).

In [ ]:
# Import modules
import torch
import numpy as np
import plotly.graph_objects as go
from tqdm.notebook import tqdm

# Select which GPU to use (if available)
gpu = 0
if torch.cuda.is_available():
    torch.cuda.set_device(gpu)
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Using device: {device}')

## Load the DeepRFSoC dataset
Ensure the DeepRFSoC dataset has been downloaded to your local drive and point to it with the variable `dataset_path` below. Consult the `README.md` in this folder on instructions on how to download the DeepRFSoC dataset.

In [ ]:
import os.path

dataset_path = './DeepRFSoC.pkl'
os.path.isfile(dataset_path) # Confirm the dataset path is correct

## Preprocess dataset
Loop over classes and noise levels (SNRs) and collect them.

In [ ]:
import pickle
# with open(dataset_path, 'rb') as f:
#     dataset = pickle.load(f)
classes = ['QPSK','BPSK','QAM16','QAM64','PSK8','PAM4','GFSK','CPFSK']
snrs = ['-20','-16','-12','-8','-4','0','4','8','12','16','20','24', '28', '30']

import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

with open(dataset_path,'rb') as f:
    Xd = pickle.load(f)
mods = classes
snrs = snrs
X = []
lbl = []
snr_lbl = []
for mod in mods:
    for snr in snrs:
        tmp = Xd[mod,snr]
        X.append(tmp)
        for i in range(Xd[mod,snr].shape[2]):
            lbl.append(mods.index(mod))
            snr_lbl.append(snr)
X = np.dstack(X)
X = np.moveaxis(X,2,0)
labels = lbl
dataset_values = X

## Create PyTorch Dataset Class
Create `Dataset` class for easy access to frames during our training loop.

In [ ]:

validation_split = 0.2
test_split = 0.1
shuffle_dataset = True
# Creating data indices for training, validation, and test splits:
dataset_size = len(labels)
indices = list(range(dataset_size))
split = int(np.floor((validation_split+test_split) * dataset_size))
if shuffle_dataset:
    np.random.seed(1234)
    np.random.shuffle(indices)
train_indices, val_test_indices = indices[split:], indices[:split]
valid_split = int(np.floor(validation_split * len(val_test_indices)))
val_indices, test_indices = val_test_indices[valid_split:], val_test_indices[:valid_split]
# Creating PT data samplers and loaders:
train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
valid_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
test_sampler = torch.utils.data.SubsetRandomSampler(test_indices)

class AMCDataset(Dataset):
    def __init__(self, dataset, labels):
        super(AMCDataset,self).__init__()
        self.classes = ['QPSK','BPSK','QAM16','QAM64','PSK8','PAM4','GFSK','CPFSK']
        self.snrs = ['-20','-16','-12','-8','-4','0','4','8','12','16','20','24','28','30']
        self.dataset, self.labels = dataset, labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        data = self.dataset[idx,:,:]
        label = self.labels[idx]
        # return data, label
        return torch.from_numpy(data.astype(np.float32).reshape(1,2,-1)), torch.tensor(label)
    
dataset = AMCDataset(dataset_values, labels)

## Define Quantised AMC Models

Four separate models quantised using Brevitas.

In [ ]:
from torch import nn
import brevitas.nn as qnn
from brevitas.quant.fixed_point import Int8ActPerTensorFixedPointMinMaxInit
class InputQuantiser(Int8ActPerTensorFixedPointMinMaxInit):
    bit_width = 16
    min_val = -2.0
    max_val = 2.0 - (2.0 ** -14)
from brevitas.quant.fixed_point import Int8WeightPerTensorFixedPoint
class Int16WeightPerTensorFixedPoint(Int8WeightPerTensorFixedPoint):
    bit_width = 16

from brevitas.quant.fixed_point import Int8ActPerTensorFixedPoint
class Int16ActPerTensorFixedPoint(Int8ActPerTensorFixedPoint):
    bit_width = 16

rt = False

class QuantAMC16w16a(nn.Module):
    def __init__(self):
        super(QuantAMC16w16a, self).__init__()
        self.conv_layers = nn.ModuleList()
        self.linear_layers = nn.ModuleList()
        self.conv_layers.append(qnn.QuantIdentity(act_quant=InputQuantiser, return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantConv2d(
            kernel_size=(1,3),
            in_channels=1,
            out_channels=64,
            bias=False,
            weight_quant=Int16WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantReLU(act_quant=Int16ActPerTensorFixedPoint, return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantConv2d(
            kernel_size=(2,3),
            in_channels=64,
            out_channels=16,
            bias=False,
            weight_quant=Int16WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantReLU(act_quant=Int16ActPerTensorFixedPoint, return_quant_tensor=True))
        self.linear_layers.append(qnn.QuantLinear(
            in_features=1984,
            out_features=128,
            bias=False,
            weight_quant=Int16WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=True))
        self.linear_layers.append(qnn.QuantReLU(act_quant=Int16ActPerTensorFixedPoint, return_quant_tensor=True))
        self.linear_layers.append(qnn.QuantLinear(
            in_features=128,
            out_features=8,
            bias=False,
            weight_quant=Int16WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=rt))
    
    def forward(self, x):
        for mod in self.conv_layers:
            x = mod(x)
        if len(x.shape) > 3:
            x = x.transpose(1,3).flatten(start_dim=1)
        else:
            x = x.transpose(0,2).flatten(start_dim=0)
        for mod in self.linear_layers:
            x = mod(x)
        return x

model = QuantAMC16w16a()

# 8-bit weight, 16-bit activation

class QuantAMC8w16a(nn.Module):
    def __init__(self):
        super(QuantAMC8w16a, self).__init__()
        self.conv_layers = nn.ModuleList()
        self.linear_layers = nn.ModuleList()
        self.conv_layers.append(qnn.QuantIdentity(act_quant=InputQuantiser, return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantConv2d(
            kernel_size=(1,3),
            in_channels=1,
            out_channels=64,
            bias=False,
            weight_quant=Int8WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantReLU(act_quant=Int16ActPerTensorFixedPoint, return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantConv2d(
            kernel_size=(2,3),
            in_channels=64,
            out_channels=16,
            bias=False,
            weight_quant=Int8WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantReLU(act_quant=Int16ActPerTensorFixedPoint, return_quant_tensor=True))
        self.linear_layers.append(qnn.QuantLinear(
            in_features=1984,
            out_features=128,
            bias=False,
            weight_quant=Int8WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=True))
        self.linear_layers.append(qnn.QuantReLU(act_quant=Int16ActPerTensorFixedPoint, return_quant_tensor=True))
        self.linear_layers.append(qnn.QuantLinear(
            in_features=128,
            out_features=8,
            bias=False,
            weight_quant=Int8WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=rt))
    
    def forward(self, x):
        for mod in self.conv_layers:
            x = mod(x)
        if len(x.shape) > 3:
            x = x.transpose(1,3).flatten(start_dim=1)
        else:
            x = x.transpose(0,2).flatten(start_dim=0)
        for mod in self.linear_layers:
            x = mod(x)
        return x

model = QuantAMC8w16a()

# 4-bit weight, 16-bit activation

from brevitas.quant.fixed_point import Int4WeightPerTensorFixedPointDecoupled

class Int4WeightPerTensorFixedPoint(Int4WeightPerTensorFixedPointDecoupled):
    bit_width = 4

class QuantAMC4w16a(nn.Module):
    def __init__(self):
        super(QuantAMC4w16a, self).__init__()
        self.conv_layers = nn.ModuleList()
        self.linear_layers = nn.ModuleList()
        self.conv_layers.append(qnn.QuantIdentity(act_quant=InputQuantiser, return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantConv2d(
            kernel_size=(1,3),
            in_channels=1,
            out_channels=64,
            bias=False,
            weight_quant=Int4WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantReLU(act_quant=Int16ActPerTensorFixedPoint, return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantConv2d(
            kernel_size=(2,3),
            in_channels=64,
            out_channels=16,
            bias=False,
            weight_quant=Int4WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantReLU(act_quant=Int16ActPerTensorFixedPoint, return_quant_tensor=True))
        self.linear_layers.append(qnn.QuantLinear(
            in_features=1984,
            out_features=128,
            bias=False,
            weight_quant=Int4WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=True))
        self.linear_layers.append(qnn.QuantReLU(act_quant=Int16ActPerTensorFixedPoint, return_quant_tensor=True))
        self.linear_layers.append(qnn.QuantLinear(
            in_features=128,
            out_features=8,
            bias=False,
            weight_quant=Int4WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=rt))
    
    def forward(self, x):
        for mod in self.conv_layers:
            x = mod(x)
        if len(x.shape) > 3:
            x = x.transpose(1,3).flatten(start_dim=1)
        else:
            x = x.transpose(0,2).flatten(start_dim=0)
        for mod in self.linear_layers:
            x = mod(x)
        return x

model = QuantAMC4w16a()

# 2-bit weight, 16-bit activation

from brevitas.quant.fixed_point import Int4WeightPerTensorFixedPointDecoupled

class Int2WeightPerTensorFixedPoint(Int4WeightPerTensorFixedPointDecoupled):
    bit_width = 2

class QuantAMC2w16a(nn.Module):
    def __init__(self):
        super(QuantAMC2w16a, self).__init__()
        self.conv_layers = nn.ModuleList()
        self.linear_layers = nn.ModuleList()
        self.conv_layers.append(qnn.QuantIdentity(act_quant=InputQuantiser, return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantConv2d(
            kernel_size=(1,3),
            in_channels=1,
            out_channels=64,
            bias=False,
            weight_quant=Int2WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantReLU(act_quant=Int16ActPerTensorFixedPoint, return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantConv2d(
            kernel_size=(2,3),
            in_channels=64,
            out_channels=16,
            bias=False,
            weight_quant=Int2WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=True))
        self.conv_layers.append(qnn.QuantReLU(act_quant=Int16ActPerTensorFixedPoint, return_quant_tensor=True))
        self.linear_layers.append(qnn.QuantLinear(
            in_features=1984,
            out_features=128,
            bias=False,
            weight_quant=Int2WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=True))
        self.linear_layers.append(qnn.QuantReLU(act_quant=Int16ActPerTensorFixedPoint, return_quant_tensor=True))
        self.linear_layers.append(qnn.QuantLinear(
            in_features=128,
            out_features=8,
            bias=False,
            weight_quant=Int2WeightPerTensorFixedPoint,
            # act_quant=Int16ActPerTensorFixedPoint,
            return_quant_tensor=rt))
    
    def forward(self, x):
        for mod in self.conv_layers:
            x = mod(x)
        if len(x.shape) > 3:
            x = x.transpose(1,3).flatten(start_dim=1)
        else:
            x = x.transpose(0,2).flatten(start_dim=0)
        for mod in self.linear_layers:
            x = mod(x)
        return x

model = QuantAMC2w16a()

# initialise all models

models = [QuantAMC16w16a(), QuantAMC8w16a(), QuantAMC4w16a(), QuantAMC2w16a()]


## Training, Validation, and Test functions

In [ ]:
from sklearn.metrics import accuracy_score

def train(model, train_loader, optimizer, criterion):
    losses = []
    # ensure model is in training mode
    model.train()    

    for (inputs, label) in tqdm(train_loader, desc="Batches", leave=False):   
        if gpu is not None:
            inputs = inputs.cuda()
            label = label.cuda()
        model.train()
        criterion.train()
        # print(f"Inputs:{inputs.shape}. label:{label}")
        # forward pass
        output = model(inputs)
        loss = criterion(output, label)
        
        # backward pass + run optimizer to update weights
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        # model.clip_weights(-1,1)
        # keep track of loss value
        losses.append(loss.cpu().detach().numpy())
           
    return losses

def valid(model, val_loader, criterion):    
    # ensure model is in eval mode
    losses = []
    model.eval() 
    y_true = []
    y_pred = []
    with torch.no_grad():
        for (inputs, label) in val_loader:
            if gpu is not None:
                inputs = inputs.cuda()
                label = label.cuda()
            output = model(inputs)
            loss = criterion(output, label)
            losses.append(loss.cpu().detach().numpy())
            pred = output.argmax(dim=1, keepdim=True)
            y_true.extend(label.tolist()) 
            y_pred.extend(pred.reshape(-1).tolist())
        
    return accuracy_score(y_true, y_pred), losses

def test(model, test_loader):    
    # ensure model is in eval mode
    model.eval() 
    y_true = []
    y_pred = []
    with torch.no_grad():
        for (inputs, label) in test_loader:
            if gpu is not None:
                inputs = inputs.cuda()
                label = label.cuda()
            output = model(inputs)
            pred = output.argmax(dim=1, keepdim=True)
            y_true.extend(label.tolist()) 
            y_pred.extend(pred.reshape(-1).tolist())
        
    return accuracy_score(y_true, y_pred)

def display_loss_plot(losses, title="Training loss", xlabel="Iterations", ylabel="Loss"):
    go.Figure([go.Scatter(y=losses)])

## Train the Quantised Models

With a cross entropy loss function and an Adam optimiser, the network is trained across 100 epochs.

**Warning!** The following cell trains **4** separate quantised models. Due to the extra quantisation limitations applied to the models, the time taken to converge to an answer is increased. This may take a while to run.

In [ ]:
import json
names = ['16w16a', '8w16a', '4w16a', '2w16a']
batch_size = 128
num_epochs = 100
patience = 8

data_loader_train = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
data_loader_valid = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)
data_loader_test = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

# save the losses for each q
training_loss = {}
validation_loss = {}

# loop over each model
for model in models:
    name = names[models.index(model)]
    if gpu is not None:
        model = model.cuda()

    # loss criterion and optimiser
    criterion = nn.CrossEntropyLoss()
    if gpu is not None:
        criterion = criterion.cuda()
    # Backup optimiser:
    optimiser = torch.optim.Adam(model.parameters(), lr=1e-4)

    running_loss = []
    running_val_loss = []
    running_test_acc = []
    min_val_loss = np.inf

    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        loss_epoch = train(model, data_loader_train, optimiser, criterion)
        val_acc, val_loss = valid(model, data_loader_test, criterion)
        print("Epoch %d: Training loss = %f, Validation loss = %f,val accuracy = %f" % (epoch, np.mean(loss_epoch), np.mean(val_loss), val_acc))
        mean_val_loss = np.mean(val_loss)
        if min_val_loss > mean_val_loss:
            print(f'Val loss decreased({min_val_loss:.6f} -> {mean_val_loss:.6f})\t Saving Model...')
            min_val_loss = mean_val_loss
            # Saving State Dict
            save_path = f'saved_models_fixed/saved_model_{name}_ch6.path'
            torch.save(model.state_dict(), save_path)
            count = 0
        else:
            if count < patience:
                count += 1
            else:
                print('Early Stopping Triggered!')
                break
        running_loss.append(np.mean(loss_epoch))
        running_val_loss.append(np.mean(val_loss))
        running_test_acc.append(val_acc)
    training_loss = running_loss
    validation_loss = running_val_loss
    with open(f'loss_metrics/training_loss_{name}.pkl','wb') as f:
        pickle.dump(training_loss,f,protocol=pickle.HIGHEST_PROTOCOL)
    with open(f'loss_metrics/validation_loss_{name}.pkl','wb') as f:
        pickle.dump(validation_loss,f,protocol=pickle.HIGHEST_PROTOCOL)

## Save Models' weights

In [ ]:
names = ['16w16a', '8w16a', '4w16a', '2w16a']

for i in range(4):
    name = names[i]
    models[i]
    save_path = f'saved_models_fixed/saved_model_{name}.path'
    torch.save(model.state_dict(), save_path)

In [ ]:
names = ['16w16a', '8w16a', '4w16a', '2w16a']
device = torch.device('cpu')
X_test = dataset_values[test_indices,:,:]
Y_test = np.array(labels)[test_indices]
accs = {}
for model in models:
    test_model = model.cpu()
    name = names[models.index(model)]
    print(f"Testing Model: {name}")
    test_model.load_state_dict(torch.load(f'saved_models_fixed/saved_model_{name}.path', map_location=device))
    test_model.eval()
    accs[name] = {}
    for snr in snrs:
        # extract classes @ SNR
        test_SNRs = list(map(lambda x: snr_lbl[x], test_indices))
        test_X_i = X_test[np.where(np.array(test_SNRs)==snr)]
        test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]
        # conf matrix
        conf = np.zeros([len(classes),len(classes)])
        confnorm = np.zeros([len(classes),len(classes)])
        # estimate classes
        in_model = np.reshape(test_X_i, (-1,1,2,128))
        for i in range(0,in_model.shape[0]):
            in_reshape = np.reshape(in_model[i,:,:,:],(-1,1,2,128))
            test_Y_i_hat = test_model(torch.from_numpy(in_model[i,:,:,:]).to(torch.float32)).cpu().detach().numpy()
            j = test_Y_i[i]
            k = int(np.argmax(test_Y_i_hat))
            conf[j,k] = conf[j,k] + 1
        for i in range(0,len(classes)):
            confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
        cor = np.sum(np.diag(conf))
        ncor = np.sum(conf) - cor
        print(f'{snr}dB SNR. Overall Accuracy: {cor / (cor+ncor)}')
        accs[name][snr] = 1.0*cor/(cor+ncor)

In [ ]:
fig = go.Figure()
for name in names:
    fig.add_trace(go.Scatter(x=snrs, y=[accs[name][snr] for snr in snrs], mode='lines+markers', name=name))

fig.update_layout(
    title="Test Accuracy vs SNR for Different Quantisation Levels",
    xaxis_title="SNR (dB)",
    yaxis_title="Accuracy",
    yaxis=dict(range=[0,1])
)
fig.show()

## Save a Test set for MATLAB

Saving a test set for the highest SNR to build hardware model in MATLAB/Simulink.

In [ ]:
# make test set of only 30dB SNR
X_test = dataset_values[test_indices,:,:]
Y_test = np.array(labels)[test_indices]
test_SNRs = list(map(lambda x: snr_lbl[x], test_indices))
test_X_i = X_test[np.where(np.array(test_SNRs)=='30')]
test_Y_i = Y_test[np.where(np.array(test_SNRs)=='30')]

from scipy.io import savemat
name = 'float_deeprfsoc'
inputs = np.reshape(test_X_i, (-1,1,2,128))
labels = test_Y_i
datadict = {'inputs': inputs, 'labels': labels}
savemat(f'matlab_saved_models/inputs_amc_{name}_30dB.mat', datadict)


## Inspect Quantised Activations in Model

In [ ]:
# Get the interlayer signal precisions
name = 'float_deeprfsoc'
model = models[0] # 16w16a
model.cpu()
model.eval()
identity = model.conv_layers[0]
conv1 = model.conv_layers[1]
relu1 = model.conv_layers[2]
conv2 = model.conv_layers[3]
relu2 = model.conv_layers[4]
dense1 = model.linear_layers[0]
relu3 = model.linear_layers[1]
dense2 = model.linear_layers[2]

# use the same input as in matlab
from scipy.io import loadmat
inputs_path = f'matlab_saved_models/inputs_amc_{name}_30dB.mat'
inputs = loadmat(inputs_path)['inputs']
inputs = torch.from_numpy(inputs).to(torch.float32)
input_frame = inputs[1,:,:,:]
input_frame.shape

inp = input_frame
print(f'Input: {inp[0,0,0:4]}')
l1 = identity(inp)
print(f'identity output {l1[0][0,0,0:4]}')
l2 = conv1(l1)
print(f'conv1 output {l2[0][0,0,0:4]}')
l3 = relu1(l2)
print(f'relu1 output {l3[0][0,0,0:4]}')
l4 = conv2(l3)
print(f'conv2 output {l4[0][0,0,0:4]}')
l5 = relu2(l4)
print(f'relu2 output {l5[0][0,0,0:4]}')
l5_flat = l5.transpose(0,2).flatten(start_dim=0)
print(f'flatten {l5_flat.shape}')
l6 = dense1(l5_flat)
print(f'dense1 output {l6[0][0:4]}')
l7 = relu3(l6)
print(f'relu3 output {l7[0][0:4]}')
l8 = dense2(l7)
print(f'dense2 output {l8}')

# output of model
output = model(inp)
print(f'model output {output}')

import math
print(f"Input - Q{int(l1.bit_width)+int(math.log2(l1.scale))}.{-int(math.log2(l1.scale))}")
print(f"Conv1 Output - Q{int(l2.bit_width)+int(math.log2(l2.scale))}.{-int(math.log2(l2.scale))}")
print(f"ReLU1 Output - Q{int(l3.bit_width)+int(math.log2(l3.scale))}.{-int(math.log2(l3.scale))}")
print(f"Conv2 Output - Q{int(l4.bit_width)+int(math.log2(l4.scale))}.{-int(math.log2(l4.scale))}")
print(f"ReLU2 Output - Q{int(l5.bit_width)+int(math.log2(l5.scale))}.{-int(math.log2(l5.scale))}")
print(f"Flatten - Q{int(l5_flat.bit_width)+int(math.log2(l5_flat.scale))}.{-int(math.log2(l5_flat.scale))}")
print(f"Dense1 Output - Q{int(l6.bit_width)+int(math.log2(l6.scale))}.{-int(math.log2(l6.scale))}")
print(f"ReLU3 Output - Q{int(l7.bit_width)+int(math.log2(l7.scale))}.{-int(math.log2(l7.scale))}")
print(f"Dense2 Output - Q{int(l8.bit_width)+int(math.log2(l8.scale))}.{-int(math.log2(l8.scale))}")
print(f"Model Output - Q{int(output.bit_width)+int(math.log2(output.scale))}.{-int(math.log2(output.scale))}")

## Inspect the Quantised Weights
We can inspect the fixed-point parameters of each set of weights, including the fractional bits learned for each layer.

In [ ]:
import math
for model in models:
    name = names[models.index(model)]
    print(f"Models: {name}")
    print(f"Conv 1 - Q{model.conv_layers[1].quant_weight().bit_width}.{-math.log2(model.conv_layers[1].quant_weight().scale)}")
    print(f"Conv 2 - Q{model.conv_layers[3].quant_weight().bit_width}.{-math.log2(model.conv_layers[3].quant_weight().scale)}")
    print(f"FC 1 - Q{model.linear_layers[0].quant_weight().bit_width}.{-math.log2(model.linear_layers[0].quant_weight().scale)}")
    print(f"FC 2 - Q{model.linear_layers[2].quant_weight().bit_width}.{-math.log2(model.linear_layers[2].quant_weight().scale)}")

## Save models' weights for MATLAB

In [ ]:
from scipy.io import savemat
names = ['16w16a', '8w16a', '4w16a', '2w16a']
for model in models:
    model = model.cpu()
    name = names[models.index(model)]
    Wconv1 = model.conv_layers[1].weight.detach().numpy()
    Wconv2 = model.conv_layers[3].weight.detach().numpy()
    Wdense1 = model.linear_layers[0].weight.detach().numpy()
    Wdense2 = model.linear_layers[2].weight.detach().numpy()
    mdict = {'Wconv1': Wconv1, 'Wconv2': Wconv2, 'Wdense1': Wdense1, 'Wdense2': Wdense2}
    savemat(f'matlab_saved_models/model_{name}.mat', mdict)
# save input for testing
inputs = np.reshape(X_test, (-1,1,2,128))
labels = Y_test
datadict = {'inputs': inputs, 'labels': labels}
savemat(f'matlab_saved_models/inputs_amc_deeprfsoc.mat', datadict)

## Save a Test set for Hardware

Saving a separate test set for hardware so that we can load this onto the RFSoC.

In [ ]:
X_test = dataset_values[test_indices,:,:]
Y_test = np.array(labels)[test_indices]
testset = {}
for snr in snrs:
    # extract classes @ SNR
    test_SNRs = list(map(lambda x: snr_lbl[x], test_indices))
    test_X_i = X_test[np.where(np.array(test_SNRs)==snr)]
    test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]
    testset[snr] = (test_X_i, test_Y_i)
import pickle
with open(f'matlab_saved_models/testset.pkl','wb') as f:
    pickle.dump(testset,f,protocol=pickle.HIGHEST_PROTOCOL)

## Inspect weights of deployed models

In [ ]:
names = ['16w16a', '8w16a', '4w16a', '2w16a']
device = torch.device('cpu')
X_test = dataset_values[test_indices,:,:]
Y_test = np.array(labels)[test_indices]
accs = {}
for model in models:
    model = model.cpu()
    name = names[models.index(model)]
    print(f"Loading model weights for {name}")
    model.load_state_dict(torch.load(f'saved_models_fixed/saved_model_{name}.path', map_location=device))
    model.eval()

---